In [1]:
import requests
requests.packages.urllib3.disable_warnings()
from bs4 import BeautifulSoup as bs
import re
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
import datetime
import pandas as pd
import os

In [2]:
browser = webdriver.Edge('D:\\edgedriver\\msedgedriver.exe')
#打开浏览器，记得先登陆


In [3]:
with open("non_rumor_url.txt",'r',encoding="utf-8") as f:
    urls = f.readlines()

In [3]:
urls = ["https://weibo.com/1608574203/K2HP9k7mO"]

In [4]:
def time_convert(t):
    # t: 2020-12-31 12:35
    if "今天" in t:
        nian = datetime.datetime.now().year
        yue = datetime.datetime.now().month
        ri = datetime.datetime.now().day
        t = "{}年{}月{}日 {}".format(nian,yue,ri,t[3:])
        timeArray = time.strptime(t, "%Y年%m月%d日 %H:%M")
    elif "月" in t:
        timeArray = time.strptime("2021年"+t, "%Y年%m月%d日 %H:%M")
    elif "刚" in t:
        return curr_time
    elif "秒前" in t:
        return curr_time - int(t[:-2])
    elif "分钟前" in t:
        return curr_time - int(t[:-3]) * 60
    elif "楼" in t:
        t = t.split(" ")
        return time_convert(t[1]+" "+t[2])
    else:
        timeArray = time.strptime(t, "%Y-%m-%d %H:%M")
    timeStamp = int(time.mktime(timeArray))
    return timeStamp


In [5]:
for url in urls:
    url = url.rstrip('\n')
    id = url.split('/')[-1]
    if not os.path.exists("non_rumor/comment/" + id + "_comment.csv"):
        
        browser.get(url + "?type=comment")
        #获取当前时间
        time.sleep(random.uniform(6,8))
        curr_time = int(time.time())
        w_id = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[2]/div[1]/div").get_attribute("id")
        if w_id[:2]!="Pl":
            print(url)
            continue
        #懒加载
        time.sleep(random.uniform(4,6))
        all_window_height =  []  # 创建一个列表，用于记录每一次拖动滚动条后页面的最大高度
        all_window_height.append(browser.execute_script("return document.body.scrollHeight;")) #当前页面的最大高度加入列表
        while True:
            browser.execute_script("scroll(0,100000)") # 执行拖动滚动条操作
            time.sleep(random.uniform(4,6))
            check_height = browser.execute_script("return document.body.scrollHeight;")
            if check_height == all_window_height[-1]:  #判断拖动滚动条后的最大高度与上一次的最大高度的大小，相等表明到了最底部
                break
            else:
                all_window_height.append(check_height) #如果不想等，将当前页面最大高度加入列表。

        #点击查看更多
        time.sleep(random.uniform(4,6))
        while True:
            try:
                watch_more = browser.find_element(By.XPATH,'//*[@id="' + w_id + '"]/div/div/div/div[4]/div/div[@class="repeat_list"]/div[2]/div/div/a')
                
                watch_more.click()
                time.sleep(random.uniform(4,6))
            except:
                break
        try:    
            top = browser.find_element_by_xpath('//*[@id="base_scrollToTop"]')
            top.click()
        except:
            pass

        #源贴：
        data1 = pd.DataFrame(columns=('uid','mid','user_name','text','parent','time'))
        source = browser.find_element_by_xpath('//*[@id="' + w_id + '"]/div/div/div')
                                     
        uid = source.get_attribute('tbinfo')
        s_mid = source.get_attribute('mid')
        uid = uid.split('=')[1]
        user_name = source.find_element_by_xpath('./div[1]/div[@class="WB_detail"]/div[1]/a[1]').text

        t = time_convert(source.find_element_by_xpath('./div[1]/div[@class="WB_detail"]/div[2]/a[1]').text)

        text = source.find_element_by_xpath('./div[1]/div[@class="WB_detail"]/div[4]').text
        d = {'uid':uid,'mid':s_mid,'user_name':user_name,'time':t,'text':text,'parent':None,"type":0}
        data1 = data1.append(d,ignore_index=True)

        #评论
        time.sleep(random.uniform(5,8))
        comment1s = browser.find_elements_by_xpath('//*[@id="' + w_id + '"]/div/div/div/div[@node-type="comment_detail"]/div/div[@class="repeat_list"]/div[@node-type="feed_list"]/div/div[@class="list_ul"]/div[@node-type="root_comment"]')

        for comment1 in comment1s: #循环
            uid = comment1.find_element_by_xpath('./div[2]/div[1]/a[1]').get_attribute('usercard')
            c_mid = comment1.get_attribute('comment_id')
            if comment1.get_attribute('node-type')=='root_comment':
                parent = s_mid
            uid = uid.split('=')[1]
            user_name = comment1.find_element_by_xpath('./div[2]/div[1]/a[1]').text
            t = time_convert(comment1.find_element_by_xpath('./div[2]/div[@class="WB_func clearfix"]/div[2]').text)
            
            text = comment1.find_element_by_xpath('./div[2]/div[1]').text
            d = {'uid':uid,'mid':c_mid,'user_name':user_name,'time':t,'text':text[text.index('：')+1:],'parent':parent,"type":1}
            data1 = data1.append(d,ignore_index = True)
            while True:
                try:
                    watch_more = comment1.find_elements_by_xpath('./div[2]/div[@class="list_box_in S_bg3"]')
                    if len(watch_more)>1:
                        watch_more = watch_more[1].find_element_by_xpath('./div/div[@node-type="more_child_comment"]/div[1]/a[@action-type="click_more_child_comment_big"]')
                    else:
                        watch_more = watch_more[0].find_element_by_xpath('./div/div[@node-type="more_child_comment"]/div[1]/a[@action-type="click_more_child_comment_big"]')
                    watch_more.click()
                    time.sleep(random.uniform(5,7))
                except:
                    break

            comment2s = comment1.find_elements_by_xpath('./div[@node-type="replywrap"]/div[@class="list_box_in S_bg3"]/div/div[@class="list_li S_line1 clearfix"]')
            for comment2 in comment2s:
                uid = comment2.find_element_by_xpath('./div/div[1]/a').get_attribute('usercard')
                mid = comment2.get_attribute('comment_id')
                parent = c_mid
                uid = uid.split('=')[1]
                user_name = comment2.find_element_by_xpath('./div/div[1]/a').text
                t = time_convert(comment2.find_element_by_xpath('./div/div[@class="WB_func clearfix"]/div[2]').text)
                text = comment2.find_element_by_xpath('./div/div[1]').text
                d = {'uid':uid,'mid':mid,'user_name':user_name,'time':t,'text':text[text.index('：')+1:],'parent':parent,"type":2}
                data1 = data1.append(d,ignore_index = True)

        data1.to_csv("non_rumor/comment/" + id +"_comment.csv",encoding='utf-8')

    if not os.path.exists("non_rumor/repost/" + id + "_repost.csv"):
        #转发
        browser.get(url + "?type=repost")
        time.sleep(random.uniform(5,8))
        curr_time = int(time.time())
        time.sleep(random.uniform(6,8))
        curr_time = int(time.time())
        w_id = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[2]/div[1]/div").get_attribute("id")
        if w_id[:2]!="Pl":
            print(url)
            continue
        source = browser.find_element_by_xpath('//*[@id="' + w_id + '"]/div/div/div')
        s_mid = source.get_attribute('mid')
        data2 = pd.DataFrame(columns=('uid','mid','user_name','text','parent','time'))
        while True:
            L_reposts = len(browser.find_elements_by_xpath('//*[@id="' + w_id + '"]/div/div/div/div[5]/div/div[@class="repeat_list"]/div[2]/div/div[@class="list_li S_line1 clearfix"]'))
            r_xpath = '//*[@id="' + w_id + '"]/div/div/div/div[5]/div/div[@class="repeat_list"]/div[2]/div/div[@class="list_li S_line1 clearfix"]'
            for r in range(L_reposts):
                uid = browser.find_elements_by_xpath(r_xpath)[r].find_element_by_xpath('./div[2]/div[1]/a[1]').get_attribute('usercard')
                mid = browser.find_elements_by_xpath(r_xpath)[r].get_attribute('mid')
                #if comment1.get_attribute('node-type')=='root_comment':
                parent = s_mid
                uid = uid.split('=')[1]
                user_name = browser.find_elements_by_xpath(r_xpath)[r].find_element_by_xpath('./div[2]/div[1]/a').text
                t = time_convert(browser.find_elements_by_xpath(r_xpath)[r].find_element_by_xpath('./div[2]/div[@class="WB_func clearfix"]/div[2]').text)
                text = browser.find_elements_by_xpath(r_xpath)[r].find_element_by_xpath('./div[2]/div[1]').text
                d = {'uid':uid,'mid':mid,'user_name':user_name,'time':t,'text':text[text.index('：')+1:],'parent':parent,"type":3}
                data2 = data2.append(d,ignore_index = True)
            try:
                page = browser.find_element_by_xpath('//*[@id="' + w_id + '"]/div/div/div/div[5]/div/div[@class="repeat_list"]/div[2]/div/div[@class="WB_cardpage S_line1"]/div[@class="W_pages"]/a[@class="page next S_txt1 S_line1"]')     
                while True:
                    browser.execute_script("scroll(0,100000)") # 执行拖动滚动条操作
                    time.sleep(random.uniform(4,6))
                    check_height = browser.execute_script("return document.body.scrollHeight;")
                    if check_height == all_window_height[-1]:  #判断拖动滚动条后的最大高度与上一次的最大高度的大小，相等表明到了最底
                        break
                    else:
                        all_window_height.append(check_height) #如果不想等，将当前页面最大高度加入列表。
           
                page.click()
                time.sleep(random.uniform(4,6))
            except:
                break

        data2.to_csv("non_rumor/repost/" + id +"_repost.csv",encoding='utf-8')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[2]/div/div[2]/div[1]/div"}
  (Session info: MicrosoftEdge=90.0.818.56)


In [8]:
if not os.path.exists("non_rumor/repost/" + "K2HP9k7mO" + "_repost.csv"):
        #转发
        browser.get(urls[0] + "?type=repost")
        time.sleep(random.uniform(5,8))
        curr_time = int(time.time())
        time.sleep(random.uniform(6,8))
        curr_time = int(time.time())
        w_id = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[2]/div[1]/div").get_attribute("id")
        if w_id[:2]!="Pl":
            print(url)
            #continue
        source = browser.find_element_by_xpath('//*[@id="' + w_id + '"]/div/div/div')
        s_mid = source.get_attribute('mid')
        data2 = pd.DataFrame(columns=('uid','mid','user_name','text','parent','time'))
        while True:
            L_reposts = len(browser.find_elements_by_xpath('//*[@id="' + w_id + '"]/div/div/div/div[5]/div/div[@class="repeat_list"]/div[2]/div/div[@class="list_li S_line1 clearfix"]'))
            r_xpath = '//*[@id="' + w_id + '"]/div/div/div/div[5]/div/div[@class="repeat_list"]/div[2]/div/div[@class="list_li S_line1 clearfix"]'
            for r in range(L_reposts):
                uid = browser.find_elements_by_xpath(r_xpath)[r].find_element_by_xpath('./div[2]/div[1]/a[1]').get_attribute('usercard')
                mid = browser.find_elements_by_xpath(r_xpath)[r].get_attribute('mid')
                #if comment1.get_attribute('node-type')=='root_comment':
                parent = s_mid
                uid = uid.split('=')[1]
                user_name = browser.find_elements_by_xpath(r_xpath)[r].find_element_by_xpath('./div[2]/div[1]/a').text
                t = time_convert(browser.find_elements_by_xpath(r_xpath)[r].find_element_by_xpath('./div[2]/div[@class="WB_func clearfix"]/div[2]').text)
                text = browser.find_elements_by_xpath(r_xpath)[r].find_element_by_xpath('./div[2]/div[1]').text
                d = {'uid':uid,'mid':mid,'user_name':user_name,'time':t,'text':text[text.index('：')+1:],'parent':parent,"type":3}
                data2 = data2.append(d,ignore_index = True)
            try:
                page = browser.find_element_by_xpath('//*[@id="' + w_id + '"]/div/div/div/div[5]/div/div[@class="repeat_list"]/div[2]/div/div[@class="WB_cardpage S_line1"]/div[@class="W_pages"]/a[@class="page next S_txt1 S_line1"]')     
                while True:
                    browser.execute_script("scroll(0,100000)") # 执行拖动滚动条操作
                    time.sleep(random.uniform(4,6))
                    check_height = browser.execute_script("return document.body.scrollHeight;")
                    if check_height == all_window_height[-1]:  #判断拖动滚动条后的最大高度与上一次的最大高度的大小，相等表明到了最底
                        break
                    else:
                        all_window_height.append(check_height) #如果不想等，将当前页面最大高度加入列表。
           
                page.click()
                time.sleep(random.uniform(4,6))
            except:
                break

        data2.to_csv("non_rumor/repost/" +"K2HP9k7mO"+"_repost.csv",encoding='utf-8')

In [59]:
data2.to_csv("non_rumor/repost/" + id +"_repost.csv",encoding='utf-8')

In [100]:
L_reposts = len(browser.find_elements_by_xpath(r_xpath))
r_xpath = '//*[@id="' + w_id + '"]/div/div/div/div[5]/div/div[@class="repeat_list"]/div[2]/div/div[@class="list_li S_line1 clearfix"]'
for r in range(L_reposts):
    print(r,L_reposts)
    print(browser.find_element_by_xpath(r_xpath))#.find_elements_by_xpath('./div[2]/div[1]/a[1]').get_attribute('usercard')
    break

0 19
<selenium.webdriver.remote.webelement.WebElement (session="dc6066034518a1e7323836d2cfc0abb1", element="09b5a0e1-fd45-4c24-b761-b1fe8ab67a37")>


In [99]:
r_xpath

'//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[5]/div/div[@class="repeat_list"]/div[2]/div/div[@class="list_li S_line1 clearfix"]'

In [23]:
browser.get("https://weibo.com/6003555098/JysY2jISR?type=comment")
#获取当前时间
time.sleep(3)
curr_time = int(time.time())

In [20]:

time.sleep(3)
all_window_height =  []  # 创建一个列表，用于记录每一次拖动滚动条后页面的最大高度
all_window_height.append(browser.execute_script("return document.body.scrollHeight;")) #当前页面的最大高度加入列表
while True:
        browser.execute_script("scroll(0,100000)") # 执行拖动滚动条操作
        time.sleep(4)
        check_height = browser.execute_script("return document.body.scrollHeight;")
        if check_height == all_window_height[-1]:  #判断拖动滚动条后的最大高度与上一次的最大高度的大小，相等表明到了最底部
            break
        else:
            all_window_height.append(check_height) #如果不想等，将当前页面最大高度加入列表。

In [21]:
#点击查看更多
while True:
    try:
        watch_more = browser.find_element(By.XPATH,'//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div/div[@class="repeat_list"]/div[2]/div/div/a')
        
        watch_more.click()
        time.sleep(random.uniform(4,6))
    except:
        break
top = browser.find_element_by_xpath('//*[@id="base_scrollToTop"]')
top.click()

In [27]:
#源贴：
data1 = pd.DataFrame(columns=('uid','mid','user_name','text','parent','time'))

source = browser.find_element_by_xpath('//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div')

uid = source.get_attribute('tbinfo')
s_mid = source.get_attribute('mid')
uid = uid.split('=')[1]
user_name = source.find_element_by_xpath('./div[1]/div[@class="WB_detail"]/div[1]/a[1]').text

t = time_convert(source.find_element_by_xpath('./div[1]/div[@class="WB_detail"]/div[2]/a[1]').text)

text = source.find_element_by_xpath('./div[1]/div[@class="WB_detail"]/div[4]').text
d = {'uid':uid,'mid':s_mid,'user_name':user_name,'time':t,'text':text,'parent':None,"type":0}
data1 = data1.append(d,ignore_index=True)


In [14]:
d

{'uid': '1688807837',
 'mid': '4581040163400699',
 'user_name': '天使芯-保护动物',
 'time': 1607688540,
 'text': '📣📣震惊世界\n坐标上海，又一起警察当街残杀走失萨摩犬，手段残忍到令人发指[发怒][发怒][发怒]\n出事地点：上海浦东上浦路165弄。\n时间：2020年12月11日上午。在小区内走失的萨摩耶二次遭到暴力毒打。第一次牙齿给打掉，第二次直接打死（有照片作证）\n\n☎️投诉电话：️02112345\n☎️浦东分局督察：02168546109\n☎️打死萨摩耶的东明路派出所电话：\n02122044671（东明派出所地址：浦东新区南林路738号）\n☎️ 投诉警察违规执法热线：12389\n\n国家农业部4499号文件有三个要求：1，要求各地政府善待流浪动物，扩建犬舍妥善安置。2，给抓捕回来的流浪狗打疫苗做绝育。3，开放领养。\n东明派出所民警严重违反国家农业部4499号文件精神，与中央政府倡导的构建文明和谐的社会背道而驰！是赤裸裸的暴力执法！一定一定要追究相关的法律责任！\n东明派出所谁给你们的权利这样滥杀无辜？？？ #上海走丢有芯片萨摩被当街打死#',
 'parent': None,
 'type': 0}

In [25]:
#评论
#comment_cnt = int(browser.find_element_by_xpath('//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div').get_attribute("count")) #评论数量
time.sleep(random.uniform(4,6))
comment1s = browser.find_elements_by_xpath('//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[@node-type="comment_detail"]/div/div[@class="repeat_list"]/div[@node-type="feed_list"]/div/div[@class="list_ul"]/div[@node-type="root_comment"]')

for comment1 in comment1s: #循环
    #try:
    #    comment1 = browser.find_element_by_xpath('//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div/div[3]/div[2]/div/div/div[' + str(num1) + ']')
    #except:
    #    break
    uid = comment1.find_element_by_xpath('./div[2]/div[1]/a[1]').get_attribute('usercard')
    c_mid = comment1.get_attribute('comment_id')
    if comment1.get_attribute('node-type')=='root_comment':
        parent = s_mid
    uid = uid.split('=')[1]
    user_name = comment1.find_element_by_xpath('./div[2]/div[1]/a[1]').text
    t = time_convert(comment1.find_element_by_xpath('./div[2]/div[@class="WB_func clearfix"]/div[2]').text)
    
    text = comment1.find_element_by_xpath('./div[2]/div[1]').text
    d = {'uid':uid,'mid':c_mid,'user_name':user_name,'time':t,'text':text[text.index('：')+1:],'parent':parent,"type":1}
    data1 = data1.append(d,ignore_index = True)
    while True:
        try:
            watch_more = comment1.find_elements_by_xpath('./div[2]/div[@class="list_box_in S_bg3"]')
            print(watch_more)
            if len(watch_more)>1:
                watch_more = watch_more[1].find_element_by_xpath('./div/div[@node-type="more_child_comment"]/div[1]/a[@action-type="click_more_child_comment_big"]')
            else:
                watch_more = watch_more[0].find_element_by_xpath('./div/div[@node-type="more_child_comment"]/div[1]/a[@action-type="click_more_child_comment_big"]')
            time.sleep(3)
            watch_more.click()
            time.sleep(random.uniform(5,7))
        except:
            break
    break
    comment2s = comment1.find_elements_by_xpath('./div[@node-type="replywrap"]/div[@class="list_box_in S_bg3"]/div/div[@class="list_li S_line1 clearfix"]')
    for comment2 in comment2s:
        uid = comment2.find_element_by_xpath('./div/div[1]/a').get_attribute('usercard')
        mid = comment2.get_attribute('comment_id')
        parent = c_mid
        uid = uid.split('=')[1]
        user_name = comment2.find_element_by_xpath('./div/div[1]/a').text
        t = time_convert(comment2.find_element_by_xpath('./div/div[@class="WB_func clearfix"]/div[2]').text)
        text = comment2.find_element_by_xpath('./div/div[1]').text
        d = {'uid':uid,'mid':mid,'user_name':user_name,'time':t,'text':text[text.index('：')+1:],'parent':parent,"type":2}
        data1 = data1.append(d,ignore_index = True)
    

[<selenium.webdriver.remote.webelement.WebElement (session="40f278a72884695e22fc1039a2f497d7", element="c1723d85-520d-41ef-aa70-4c6ac0968712")>]
[<selenium.webdriver.remote.webelement.WebElement (session="40f278a72884695e22fc1039a2f497d7", element="c1723d85-520d-41ef-aa70-4c6ac0968712")>]
[<selenium.webdriver.remote.webelement.WebElement (session="40f278a72884695e22fc1039a2f497d7", element="c1723d85-520d-41ef-aa70-4c6ac0968712")>]


In [ ]:
data1

In [6]:
#转发
browser.get('https://weibo.com/6003555098/JysY2jISR?type=repost')
time.sleep(random.uniform(4,6))
curr_time = int(time.time())

In [7]:
data2 = pd.DataFrame(columns=('uid','mid','user_name','text','parent','time'))
while True:
    reposts = browser.find_elements_by_xpath('//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[5]/div/div[@class="repeat_list"]/div[2]/div/div[@class="list_li S_line1 clearfix"]')
    for repost in reposts:
        uid = repost.find_element_by_xpath('./div[2]/div[1]/a[1]').get_attribute('usercard')
        mid = repost.get_attribute('mid')
        #if comment1.get_attribute('node-type')=='root_comment':
        parent = s_mid
        uid = uid.split('=')[1]
        user_name = repost.find_element_by_xpath('./div[2]/div[1]/a').text
        t = time_convert(repost.find_element_by_xpath('./div[2]/div[@class="WB_func clearfix"]/div[2]').text)
        text = repost.find_element_by_xpath('./div[2]/div[1]').text
        d = {'uid':uid,'mid':mid,'user_name':user_name,'time':t,'text':text[text.index('：')+1:],'parent':parent,"type":3}
        data2 = data2.append(d,ignore_index = True)
    try:
        page = browser.find_element_by_xpath('//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[5]/div/div[@class="repeat_list"]/div[2]/div/div[@class="WB_cardpage S_line1"]/div[@class="W_pages"]/a[@class="page next S_txt1 S_line1"]')
        page.click()
        time.sleep(random.uniform(4,6))
    except:
        break
        
    
    

In [33]:
data1.to_csv(s_mid+"_comment.csv",encoding='utf-8')

In [42]:
data2.to_csv(s_mid+"_repost.csv",encoding='utf-8')

In [ ]:
----------------------------------------------------------------------------------------------

In [83]:
data2

,uid,mid,user_name,text,parent,time,type
0,5079202686,4495141022901027,liuwentingchina,牛掰牛掰 有这么多记录照片都可以这样,4494815544688017,1587208560,3.0
1,5772668165,4495140905560250,EnAdage,打的就是你这种人，可真不要脸啊//@Sirizjrng:牛逼牛逼这都能打拳,4494815544688017,1587208560,3.0
2,2787493700,4495140883882787,摩诃有量-风中吃药,//@兔唧唧_ :我之前很注意广州地铁的宣传图，大概是三男一女或者五男二女的比例为什么我这么...,4494815544688017,1587208560,3.0
3,5129300878,4495140788243852,最终致意,无fk说,4494815544688017,1587208500,3.0
4,5887160511,4495140699990053,cocoluna_,//@陈猿猿李狮狮:又转发到了我这里//@幸运的小五在上海: 没有田也没有园//@满城狂草:...,4494815544688017,1587208500,3.0
...,...,...,...,...,...,...,...
135,6132884038,4495134395371636,富贵儿富贵儿鸭oao,//@龙猫自在://@故园湾里://@陈猿猿李狮狮:又转发到了我这里//@幸运的小五在上海:...,4494815544688017,1587207000,3.0
136,3032306195,4495134378512972,龙大头,贱不贱？超贱！//@陈猿猿李狮狮:又转发到了我这里//@幸运的小五在上海: 没有田也没有园/...,4494815544688017,1587207000,3.0
137,1779851063,4495134311321970,酒窝宝宝甜甜甜,//@胖次胖次君://@十一点之前必睡觉://@代表人类给你一个吻://@陈猿猿李狮狮:又转...,4494815544688017,1587206940,3.0
138,3225834805,4495134287042050,黑猫白猫猫花猫,为什么//@沈夜焰://@卡李://@一生不羁要上天://@裴图龙云:这就是赤裸裸得在抢功！...,4494815544688017,1587206940,3.0


In [106]:
#爬取谣言url
browser.get("https://service.account.weibo.com/index?type=5&status=0&page=1")# + str(page))

for page in range(1,501):
    print("page=",page)
    browser.get("https://service.account.weibo.com/index?type=5&status=0&page=" + str(page))
    time.sleep(random.uniform(3,6))
    mid = []
    content = bs(browser.page_source,"html.parser",from_encoding='utf-8')
    a= content.select("#pl_service_showcomplaint tr a")[::3]
    for i in range(20):
        click_url = a[i].attrs['href']
        browser.get("https://service.account.weibo.com" + click_url)
        time.sleep(random.uniform(3,6))
        try:
            f = browser.find_element(By.XPATH,'//*[@id="pl_service_common"]/div[4]/div[2]/div/div/div/div/p/a')
        except:
            continue
        mid.append(f.get_attribute('href'))
    with open("url.txt","a",encoding="utf-8") as f:
        mid = "\n".join(mid) + '\n'
        f.writelines(mid)
    



page= 1
page= 2
page= 3
page= 4
page= 5
page= 6
page= 7
page= 8
page= 9
page= 10
page= 11
page= 12
page= 13
page= 14
page= 15
page= 16
page= 17
page= 18
page= 19
page= 20
page= 21
page= 22
page= 23
page= 24
page= 25
page= 26
page= 27
page= 28
page= 29
page= 30
page= 31
page= 32
page= 33
page= 34
page= 35
page= 36
page= 37
page= 38
page= 39
page= 40
page= 41
page= 42
page= 43
page= 44
page= 45
page= 46
page= 47
page= 48
page= 49
page= 50
page= 51
page= 52
page= 53
page= 54
page= 55
page= 56
page= 57
page= 58
page= 59
page= 60
page= 61
page= 62
page= 63
page= 64
page= 65
page= 66
page= 67
page= 68
page= 69
page= 70
page= 71
page= 72
page= 73
page= 74
page= 75
page= 76
page= 77
page= 78
page= 79
page= 80
page= 81
page= 82
page= 83
page= 84
page= 85
page= 86
page= 87
page= 88
page= 89
page= 90
page= 91
page= 92
page= 93
page= 94
page= 95
page= 96
page= 97
page= 98
page= 99
page= 100
page= 101
page= 102
page= 103
page= 104
page= 105
page= 106
page= 107
page= 108
page= 109
page= 110
page= 11

IndexError: list index out of range

In [62]:
a = browser.find_element_by_xpath('//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div/div[3]/div[2]/div/div/div[1]')


In [68]:
a.find_element_by_xpath('./div[2]/div[1]/a[1]').text

'天使芯-保护动物'

In [ ]:
[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div/div[3]/div[2]/div/div/div[1]
[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div/div[3]/div[2]/div/div/div[1]/div[2]/div[5]/div/div[1]/div/a[2]
[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div/div[3]/div[2]/div/div/div[1]/div[2]/div[5]/div/div[31]/div/a
//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div/div[3]/div[2]/div/div/div[1]/div[2]/div[1]/a[1]
//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div/div[3]/div[2]/div/div/div[1]/div[2]/div[3]/div[2]
//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div/div[3]/div[2]/div/div/div[1]/div[2]/div[1]
//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div/div[3]/div[2]/div/div/div[1]/div[2]/div[1]/text()


In [20]:
browser.find_element_by_xpath('//*[@id="Pl_Official_WeiboDetail__73"]/div/div/div/div[4]/div/div[3]/div[2]/div/div/div[29]/div[2]/div[5]')

<selenium.webdriver.remote.webelement.WebElement (session="e94abfffd909873c73451a5a2d670e03", element="2b04b99d-bce0-49c0-8d7f-ee543a5f1feb")>

'斩幽冥：是挺残忍，可是根源在警察吗？爱狗人士有心拍照片，为啥不先救下来，等他主人来找呢？那个狗主人为啥把狗弄丢呢？ 警察合理执法，你们这些爱狗人士有意见？这么牛逼，怎么不去堵警察局啊？网上当什么云爱狗人士呢'

In [35]:
timeStamp

1381419600